# Optional: House Prices using Regression

This notebook will focus on the application of Neural Networks for house price prediction. Instead of images as pixel-wise inputs, we will consider generic values that describe various features for houses.

![teaser](images/teaser.jpg)

In our previous exercises we omitted a detailed data preperation overview, as it was not the main focus of those exercises since it was disctracting your attention from the basics of neural networks. In this notebook, we will optionally look a little more in-depth in data analysis. In the end, the class will not focus tremendously on these tasks but they will be essential routines that you will encounter if you choose to work in the area of deep/machine learning.

The task itself is a *regression* problem and we will give you a few hinters how you can extend your build deep learning library of the first notebook to solve it, but you can also explore it as a *classification* task.

<div class="alert alert-danger">
    <h3>Warning</h3>
    <p>We will not explore neural network implementations in this exercise. If you want to train a network in the end, we suggest to work on "1_FullyConnectedNets.ipynb" first before starting the neural network part of this submission as we will be using the fully connected neural network class of the mentioned notebook.</p>
</div>

In [1]:
# As usual, a bit of setup

import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

# House Price Data
## Exploration

Make sure to run the *download_datasets.sh* script first before running the upcoming cell. Previously, we provided you with a data loading wrapper function to access the CIFAR10 data. This time around, our input is a csv file which we will load ourselves using [pandas](https://pandas.pydata.org) where we can easily access and alter entries in our data matrix. Let's have a small glimpse how the data looks like!

In [2]:
# Load the data
data = pd.read_csv("datasets/house_prices_data.csv")
labels = pd.read_csv("datasets/house_prices_labels.csv")

In [3]:
#You can easily get an overview of our features using .info(). Note that not all features are actually numbers!
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 79 columns):
MSSubClass       1000 non-null int64
MSZoning         1000 non-null object
LotFrontage      827 non-null float64
LotArea          1000 non-null int64
Street           1000 non-null object
Alley            64 non-null object
LotShape         1000 non-null object
LandContour      1000 non-null object
Utilities        1000 non-null object
LotConfig        1000 non-null object
LandSlope        1000 non-null object
Neighborhood     1000 non-null object
Condition1       1000 non-null object
Condition2       1000 non-null object
BldgType         1000 non-null object
HouseStyle       1000 non-null object
OverallQual      1000 non-null int64
OverallCond      1000 non-null int64
YearBuilt        1000 non-null int64
YearRemodAdd     1000 non-null int64
RoofStyle        1000 non-null object
RoofMatl         1000 non-null object
Exterior1st      1000 non-null object
Exterior2nd      1000 non-nu

In [4]:
# Using the describe function we can get an overview about numerical ranges
data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1000.000000,827.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,993.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.0000,1000.00000,1000.000000,1000.000000,1000.000000
mean,56.960000,70.088271,10582.854000,6.135000,5.566000,1971.083000,1984.754000,109.542800,456.256000,42.119000,...,477.310000,97.492000,47.07300,22.154000,4.078000,14.6840,2.77100,32.134000,6.216000,2007.829000
std,42.395233,24.727977,10423.604539,1.388057,1.118434,30.498234,20.710925,182.186082,474.339455,150.778045,...,217.629834,126.267618,64.82921,62.559267,33.389771,55.1937,39.32069,323.319126,2.722638,1.336992
min,20.000000,21.000000,1300.000000,1.000000,2.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.00000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7530.000000,5.000000,5.000000,1953.000000,1967.000000,0.000000,0.000000,0.000000,...,334.500000,0.000000,0.00000,0.000000,0.000000,0.0000,0.00000,0.000000,4.000000,2007.000000
50%,50.000000,70.000000,9475.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,385.000000,0.000000,...,480.000000,0.000000,28.00000,0.000000,0.000000,0.0000,0.00000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11640.500000,7.000000,6.000000,2001.000000,2004.000000,176.000000,738.250000,0.000000,...,588.000000,171.000000,68.00000,0.000000,0.000000,0.0000,0.00000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,5644.000000,1085.000000,...,1418.000000,736.000000,547.00000,552.000000,508.000000,480.0000,648.00000,8300.000000,12.000000,2010.000000


In [5]:
# Our target variable is the SalesPrice
# The pandas dataframe string it is data_train
# We explore it here
labels['SalePrice'].describe()

count      1000.000000
mean     182814.417000
std       81736.545419
min       35311.000000
25%      130000.000000
50%      165000.000000
75%      215000.000000
max      755000.000000
Name: SalePrice, dtype: float64

We will apply operations to a small subset of our data first.

In [6]:
# Create smaller test data frames
data_small = data[:5]
labels_small = labels[:5]

## Transforms

In comparison to the previous notebooks, we always provided the data and label transforms for your convenience. However, if you want to apply your model to real life scenarios, you will have to provide a transform function that prepares raw data for your network. We already wrote a short transform class for you which will provide initial data augmenations for you under **exercise_code/transforms.py**. However, if you want to improve your network's performance, you will have to edit this class and change it to your liking!

In the end, you will not only submit your network model file but als your transform class! Using this class we can alter the test data as well as labels so that your neural network can correctly classify it.

In [7]:
# Import the transform class
from exercise_code.transforms import Transforms

## Missing Data & Non-Numerical Values

Real life data is usually not perfect. There might be unreasonable or even missing entries. Let us first check out missing entries in our dataset.

In [8]:
# Explore missing data
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# Only show top 20 entries
missing_data[:20]

,Total,Percent
PoolQC,995,0.995
MiscFeature,969,0.969
Alley,936,0.936
Fence,796,0.796
FireplaceQu,447,0.447
LotFrontage,173,0.173
GarageCond,56,0.056
GarageType,56,0.056
GarageYrBlt,56,0.056
GarageFinish,56,0.056


As we have seen before, there are some columns that contains string information which can not parsed that easily. In order to use this information, one has to transform them into a (for the network) read-able format. 

For the initial setup, we will do the following two steps:
- we will omit non-numerical columns
- we will set all numerical missing values to 0

Those decisions are obviously not optimal. You are free to explore how one should use non-numerical entries to your advantage.

<div class="alert alert-info">
    <h3>Inline Question</h3>
    <p>What solution would you propose to handle missing numerical values? Do you think we can just transform non-numerical values to integers or should one do something more elaborate? In addition to that there might be new non-numerical entries that are not present in the training set. What would you propose how we should handle those?</p>
    <p>**Your answer:** </p>
</div>

In [9]:
# For this notebook, we are performing a regression here for numerical attributes only
data_small = Transforms.get_only_numeric_attributes(data_small, verbose=True)
labels_small = Transforms.get_only_numeric_attributes(labels_small, verbose=True)

Shape of the processed data with numerical features: (5, 36)
Shape of the processed data with numerical features: (5, 1)


As you can see, we lost quite a bit of our features and are only left with 36 features.

## Normalization

As we have seen in our batch normalization notebook, it is of utmost important to properly normalize the input values for our network. We use our provided function.

In [10]:
# Based on the min and max values that were learned on the training set,
# we are going to apply the same on the validation set
data_small = Transforms.min_max_scalar(data_small, invoker='data')
labels_small = Transforms.min_max_scalar(labels_small, invoker='label')

## Finished Transform Class

Our final transform class should have a single call to transform data as well as labels. This function will be called by our test as well and you should change it if you want to apply other transforms.

In [11]:
# Create  a new small test set first
data_small = data[:5]
labels_small = labels[:5]

In [12]:
# Check the full transform function for data as well as labels
prepared_data_small = Transforms.apply_data_transforms(data_small)
prepared_labels = Transforms.apply_labels_transforms(labels_small)

In [13]:
# Lets have a short look to compare our prepared labels
print("Original labels:", labels_small)
print("Normalized labels:", prepared_labels)

Original labels:    SalePrice
0     176485
1     385000
2     395000
3     230000
4     157000
Normalized labels: [[0.08186975]
 [0.95798319]
 [1.        ]
 [0.30672269]
 [0.        ]]


## Data split

As for every training task we have to split up our provided data to validate our trained models.

In [14]:
# Perform the split on data to create train and validation set
X_train = data[:800]
X_val = data[800:]
y_train = labels[:800]
y_val = labels[800:]
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(800, 79) (200, 79) (800, 1) (200, 1)


Check out the **min_max_scalar** function of the **Transforms** class. This function scales all values between zero and one as one can do for images. The difference here is that -- while images have a fixed minimum and maximum in 0 and 255 -- we don't know the theoretical minima and maxima for your data and thus our function is evoked differently for the training and validation/test set. 

You have to take those things into account for all transforms you'd like to implement!

In [15]:
X_train = Transforms.apply_data_transforms(X_train, mode='train')
y_train = Transforms.apply_labels_transforms(y_train, mode='train')

In [16]:
# Shapes of the training data
X_train.shape, y_train.shape

((800, 36), (800, 1))

In [17]:
# Convert the validation data using our now initialized transform class
X_val = Transforms.apply_data_transforms(X_val,mode='val')
y_val = Transforms.apply_labels_transforms(y_val, mode='val')

In [18]:
X_val.shape, y_val.shape

((200, 36), (200, 1))

Phew, that was a lot of work, but we are now done with our data preperation for now and can move to the actual network part!

# Next _optional_ steps:

## Classification of Mean

We are now ready to train networks. As a first step, you can simplify the task to a simple classification task that we can solve using our network structure from the first notebook.

In order to to this, we have to use different labels. We will alter our labels such that we return 1 if the entry is bigger than the mean of our training set or 0 if it is smaller. This can be done easily with our transform function.

### Data Preparation

In [19]:
# Transform the labels into binary values since this is a classification task
def convert_to_binary_label(input_vector):
    """
    :param input_vector: a vector of real numbers
    :return: vector of 0,1 depending upon if the value is greater than mean value of the vector
    """
    mean_value = np.mean(input_vector)
    label_vector = np.array(input_vector > mean_value)
    return label_vector.astype(int)

In [20]:
y_train_binary = convert_to_binary_label(y_train)
y_val_binary = convert_to_binary_label(y_val)

In [23]:
# Convert values into a dictionary of the required format
# This dictionary can be fed into the NN
input_data = Transforms.prepare_dictionary(X_train, X_val, y_train_binary, y_val_binary)

{'X_train': array([[0.        , 0.23322684, 0.03595784, ..., 0.        , 0.63636364,
         0.25      ],
        [0.        , 0.36421725, 0.06311435, ..., 0.        , 0.45454545,
         0.5       ],
        [0.        , 0.3514377 , 0.0604174 , ..., 0.        , 0.54545455,
         0.25      ],
        ...,
        [0.        , 0.22683706, 0.0370656 , ..., 0.        , 0.81818182,
         0.        ],
        [0.58823529, 0.11821086, 0.00942298, ..., 0.        , 0.27272727,
         0.        ],
        [0.        , 0.23961661, 0.04059922, ..., 0.        , 0.09090909,
         0.25      ]]), 'y_train': array([[0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0]

### Network

Now it is your turn! Initialize a neural network as in the first notebook using the **FullyConnectedNet** class that you wrote prior. This task should not be that hard but as a sanity check you should first try to overfit on a smaller data first.

In [27]:
# Import the previously used solver and network classes
from exercise_code.solver import Solver
from exercise_code.networks.fc_net import *
learning_rate = [1e-3]
model = FullyConnectedNet([10], input_dim=36, num_classes=1, weight_scale=5e-2, dtype=np.float64)
solver = Solver(model, input_data,
                            print_every=100, 
                            num_epochs=15, 
                            batch_size=250,
                            update_rule='adam',
                            verbose=True,
                            optim_config={
                                'learning_rate': learning_rate,
                            })
solver.train()


IndexError: index 1 is out of bounds for axis 1 with size 1

## A first look at: Regression

Previously, we approached the problem similarly to our previous tasks, i.e., we considered it like a classification problem. We can approximate the actual value by using more fine grained buckets but we have a hard time predicting an actual value.

For this task, we will explore regression to directly predict the actual numerical value. We have to make some changes to our loss function to use the **l2 loss**. Please take a look at the `l2_loss` function in `layers.py` for the implementation.

There are also some updates for the solver. As we directly predict float values, there is no easy notion of "accuracy". Thus, we will only consider the loss value and look for the model with the smallest loss.  Please take a look at `update_accuracy` function in `solver.py`.

In [28]:
# Convert values into a dictionary of the required format
# This dictionary can be fed into the NN
input_data_for_NN = Transforms.prepare_dictionary(X_train, X_val, y_train, y_val)

In [33]:
# ...
from exercise_code.solver import Solver
from exercise_code.networks.fc_net import *
learning_rate = [1e-2]
model = FullyConnectedNet([10], input_dim=36, num_classes=1, weight_scale=5e-2, loss_function='l2', dtype=np.float64)
solver = Solver(model, input_data_for_NN,
                            print_every=100, 
                            num_epochs=15, 
                            batch_size=25,
                            update_rule='adam',
                            verbose=True,
                            optim_config={
                                'learning_rate': learning_rate,
                            })
solver.train()

(Iteration 1 / 480) loss: 1.222048
(Epoch 0 / 15) train loss: 0.182899; val_loss: 0.166887


TypeError: can't multiply sequence by non-int of type 'float'

## Save the model

When you are satisfied with your training, you can save the model but there will be no submission for this notebook. The required number of submissions for this class have been reduced to 6 from 7.

<div class="alert alert-danger">
    <h3>Warning</h3>
    <p>You might get an error like this:</p>
    <p>PicklingError: Can't pickle <class 'exercise_code.classifiers.softmax.SoftmaxClassifier'>: it's not the same object as exercise_code.classifiers.softmax.SoftmaxClassifier</p>
    <p>The reason is that we are using autoreload and working on this class during the notebook session. If you get this error simply restart the kernel and rerun the whole script (Kernel -> Restart & Run All) or only the important cells for generating your model.</p>
</div>

In [ ]:
from exercise_code.model_savers import save_fully_connected_net
save_fully_connected_net(best_model, modelname='house_prices')